In [3]:
import pandas as pd

In [71]:
movies = pd.read_csv(r'G:\My Drive\comp_sci\AI\CS50 Intro to AI\Search\degrees\large\movies.csv')
people = pd.read_csv(r'G:\My Drive\comp_sci\AI\CS50 Intro to AI\Search\degrees\large\people.csv')
stars = pd.read_csv(r'G:\My Drive\comp_sci\AI\CS50 Intro to AI\Search\degrees\large\stars.csv')

In [72]:
people.columns = ['person_id', 'name', 'birth']
movies.columns = ['movie_id', 'title', 'year']

In [73]:
stars = stars.merge(people, 'left', 'person_id').merge(movies, 'left', 'movie_id')
stars = stars[['name', 'title']]
stars.columns = ['actor', 'movie']

In [78]:
class Node:

    def __init__(self, movie, cast, parent_movie, parent_actor):
        self.movie = movie
        self.cast = cast
        self.parent_movie = parent_movie
        self.parent_actor = parent_actor

class Network:

    def __init__(self, actor, stars):
        self.actor = actor
        self.stars = stars
        self.frontier = [Node(movie,
                              list(self.stars.groupby('movie').get_group(movie)['actor']), 
                              None,
                              None) 
                         for movie in stars.groupby('actor').get_group(actor)['movie']]

            
    def find_degrees(self, target_actor):
        # Returns a list with tuples (movie, actor) 
        # linking the Network actor to the target actor
        self.explored = set()
        for movie in self.frontier:
            self.explored.add(movie.movie)

        solution = []
        while solution == []:
            try:    
                explored_movie = self.frontier.pop(0)
            except:
                raise Exception('No connection found between actors')
            neighbor_movies = []
            for actor in explored_movie.cast:
                for movie in self.stars.groupby('actor').get_group(actor)['movie']:
                    if movie not in self.explored:
                        self.explored.add(movie)
                        neighbor_movies.append(Node(movie,
                                                    list((self.stars.groupby('movie').get_group(movie)['actor'])),
                                                    explored_movie,
                                                    actor))
            for movie in neighbor_movies:
                for actor in explored_movie.cast:
                    if actor == target_actor:
                        parent_movie = explored_movie
                        parent_actor = actor
                        while parent_movie is not None:
                            solution.append((parent_movie.movie, parent_actor))
                            parent_actor = parent_movie.parent_actor
                            parent_movie = parent_movie.parent_movie
                        solution.reverse()
                        print(f'{self.actor} is {len(solution)} {'degree' if len(solution) == 1 else 'degrees'} from {target_actor}.')
                        print(f'{self.actor} starred in {solution[0][0]} with {solution[0][1]}.')
                        for i in range(1, len(solution)):
                            print(f'{solution[i-1][1]} starred in {solution[i][0]} with {solution[i][1]}')
                        break
                self.frontier.append(movie)

In [77]:
network = Network('Dustin Hoffman', stars)

In [ ]:
network.find_degrees('Jack Nicholson')

Dustin Hoffman is 2 degrees from Jack Nicholson.
Dustin Hoffman starred in Rain Man with Tom Cruise.
Tom Cruise starred in A Few Good Men with Jack Nicholson


In [81]:
stars.groupby('movie').get_group('Rain Man')

,actor,movie
82011,Dustin Hoffman,Rain Man
82012,Tom Cruise,Rain Man
82013,Valeria Golino,Rain Man
82014,Gerald R. Molen,Rain Man
